In [11]:
# import modules
from datetime import datetime
import pandas as pd
import yfinance as yf

IMPORTARE DATA FROM API QUANDL

url = https://data.nasdaq.com/account/profile
api key = 'MY8qCRZ5-R6YNMqoz_kK'

PRE-PROCESSING

WE START DOWNLOADING

In [1]:
#TO DO aggiungere il benchmark, e list of letters

def downloader(country, category, start_day, last_day, number_of_stocks_to_download, stock_selected, ben, benchmarks_dictionary):
    df_tickers = pd.read_excel("INPUT/yticker.xlsx")
    
    df_tickers = df_tickers.rename(columns = {"Unnamed: 1": "Name", "Unnamed: 3": "Industry", "Unnamed: 4":"Country"})
    
    df_tickers = df_tickers.iloc[3:][:]
    
    if country != '':
        df_tickers = df_tickers[df_tickers["Country"] == country]
    if category != '':
        df_tickers = df_tickers[df_tickers["Industry"] == category]
    if stock_selected != '':
        df_tickers = df_tickers[df_tickers["Yahoo Stock Tickers"] == stock_selected]
        print("filtrato")
        print(df_tickers.head(2))

    lista_tickers = df_tickers.iloc[:][["Yahoo Stock Tickers"]]

    
    start_time = datetime.now()
    
    df_stocks_total = pd.DataFrame()
    
    counter_lista = 0
    counter_lista_corretti= 0
    
    lista_tickers_filtrato = lista_tickers[:][:number_of_stocks_to_download]
    
    print("I am going to download:", len(lista_tickers_filtrato), "stocks.") 
    print("\nFor the time range:", start_day, " ", last_day)
    
    
    key_list = list(benchmarks_dictionary.keys())
    val_list = list(benchmarks_dictionary.values())
    position = val_list.index(ben)
    ben_word = key_list[position]
    
    
    print("\nBenchmark is:", ben_word)
    if country != '': 
        print("\nCountry selected:", country)
    if category != '': 
        print("\nCategory selected:", category)
    if stock_selected != '': 
        print("\nStock selected:", stock_selected)
    
        
    for index, row in lista_tickers_filtrato.iterrows():
        ticker= row['Yahoo Stock Tickers'] 
        counter_lista +=1
    # get the data in dataframe format
        stock = yf.download(ticker, start = start_day, end = last_day, progress=False)
    #add a column with the ticker name
        stock["ticker"] = ticker
    #filtering only needed columns
        stock = stock[["ticker", "Close", "Volume"]]
        df_stocks_total = pd.concat([df_stocks_total, stock])        
        if counter_lista%5== 0: 
        
            print("I am downloading:", ticker, ".This is number:", counter_lista,"out of:", len(lista_tickers_filtrato),".",
            "\nSo far I have downloaded properly:", df_stocks_total["ticker"].nunique(), "stocks")
     
    benchmark_data = yf.download(ben, start = start_day, end = last_day, progress=False)
    benchmark_data["ticker"] = ben
    benchmark_data = benchmark_data[["ticker", "Close", "Volume"]]
    df_stocks_total = pd.concat([df_stocks_total, benchmark_data])    
    
    df_stocks_total_groupby = df_stocks_total.groupby("ticker").count()
    
    print("We have a total of:", len(df_stocks_total_groupby[df_stocks_total_groupby["Close"]> 1]), "correctly and with multiple data entries")  
    
    end_time = datetime.now()  
    print('Duration: {}'.format(end_time - start_time))
    
    df_stocks_total.to_csv('INPUT/stocks_downloaded.csv')

    
    return df_stocks_total, ben
